# Generate bug reports for runtime crashes

We limit to one runtime crash per testbed and program pair. This is becuase the underlying problem may be parameter-insensitive.

In [1]:
import db as db
import sqlalchemy as sql
from db import *
%run uti tl.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

In [2]:
TABLE_NAME = "CLSmith w. cl_launcher"
TABLE = cl_launcherCLgenResult

In [15]:
from collections import defaultdict
from labm8 import fs
from io import StringIO

from reproduce import reproduce

fs.mkdir("outbox")
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.classification == "Runtime crash",
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id
        if key in errs:
            result.dupe = 1
            continue
        errs.add(key)
        
        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        
        outpath = fs.path("outbox", f"bug-report-{vendor}-{result.id}.sh")

        buf = StringIO()
        reproduce(**{
            "result_id": result.id,
            "report": "c",
            "file": buf
        })

        with open(outpath, "w") as outfile:
            print(buf.getvalue(), file=outfile)
        result.submitted = 1
        print(outpath)
print("done")